In [3]:
# validate_data.py  ←  FULL VALIDATION

import sqlite3
import pandas as pd
import logging

DB = r"E:\work\DEPI\graduation promax\data\warehouse_final\ivf_star_schema.db"
LOG = r"E:\work\DEPI\graduation promax\src\ETL\logs\validation_log.txt"

logging.basicConfig(
    filename=LOG,
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s"
)

# 1 = OK, -1 = مشاكل
STATUS_FLAG = 1


def run_validation():
    global STATUS_FLAG
    conn = sqlite3.connect(DB)

    print("\n🔎 RUNNING FULL VALIDATION...\n")

    # -----------------------------
    # 1) Check every table exists
    # -----------------------------
    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table'",
        conn
    )
    logging.info(f"Tables found:\n{tables}")
    print("🧾 Tables in DB:\n", tables, "\n")

    # -----------------------------
    # 2) Check duplicated rows
    # -----------------------------
    for tbl in tables["name"]:
        df = pd.read_sql(f"SELECT * FROM {tbl}", conn)
        dup_count = df.duplicated().sum()
        if dup_count > 0:
            STATUS_FLAG = -1
        print(f"🔍 Duplicates in {tbl} → {dup_count}")
        logging.info(f"Duplicates in {tbl}: {dup_count}")

    # -----------------------------
    # 3) Check NULL values
    # -----------------------------
    print("\n🕳 NULL VALUES CHECK:")
    for tbl in tables["name"]:
        df = pd.read_sql(f"SELECT * FROM {tbl}", conn)
        nulls = df.isnull().sum()
        if nulls.sum() > 0:
            STATUS_FLAG = -1
        print(f"\n⚠ Table: {tbl}\n{nulls}")
        logging.info(f"NULLS in {tbl}:\n{nulls}")

    # -----------------------------
    # 4) Check invalid FK relations
    # -----------------------------
    fk_checks = {
        # fact_ivf_cycle → dim_time
        "Invalid FK: fact_ivf_cycle → dim_time": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_ivf_cycle fc
            LEFT JOIN dim_time dt 
                ON fc.cycle_start_time_id = dt.time_id
            WHERE fc.cycle_start_time_id IS NOT NULL   -- NULL مسموح
              AND dt.time_id IS NULL
        """,

        # fact_transfer → dim_time
        "Invalid FK: fact_transfer → dim_time": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer ft
            LEFT JOIN dim_time dt 
                ON ft.transfer_time_fk = dt.time_id
            WHERE ft.transfer_time_fk IS NOT NULL      -- NULL مسموح
              AND dt.time_id IS NULL
        """,

        # fact_transfer → dim_doctor
        "fact_transfer → dim_doctor": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer f
            LEFT JOIN dim_doctor d 
                ON f.doctor_id = d.doctor_id
            WHERE f.doctor_id IS NOT NULL
              AND d.doctor_id IS NULL
        """,

        # fact_transfer_embryo → fact_transfer
        "fact_transfer_embryo → fact_transfer": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer_embryo fe
            LEFT JOIN fact_transfer ft 
                ON fe.transfer_sk = ft.transfer_sk
            WHERE fe.transfer_sk IS NOT NULL
              AND ft.transfer_sk IS NULL
        """,

        # fact_transfer_embryo → dim_embryo
        "fact_transfer_embryo → dim_embryo": """
            SELECT COUNT(*) AS invalid_fk
            FROM fact_transfer_embryo fe
            LEFT JOIN dim_embryo e 
                ON fe.embryo_id = e.embryo_id
            WHERE fe.embryo_id IS NOT NULL
              AND e.embryo_id IS NULL
        """
    }

    print("\n🔗 FOREIGN KEY CHECKS:")
    for title, q in fk_checks.items():
        df_fk = pd.read_sql(q, conn)
        invalid_fk_count = int(df_fk.iloc[0, 0])
        if invalid_fk_count > 0:
            STATUS_FLAG = -1
        print(f"{title} → {invalid_fk_count}")
        logging.info(f"{title}: {invalid_fk_count}")

    # -----------------------------
    # 5) Check ID formats (Regex)
    # -----------------------------
    print("\n🧬 ID FORMAT CHECK:")
    id_checks = {
        "female_id": r"^f_\d+$",
        "male_id": r"^m_\d+$",
        "protocol_id": r"^prot_.*",
        "embryo_id": r"^emb_.*",
        "outcome_id": r"^out_.*",
        "doctor_id": r"^dr_.*",
    }

    for tbl in tables["name"]:
        df = pd.read_sql(f"SELECT * FROM {tbl}", conn)
        for col, pat in id_checks.items():
            if col in df.columns:
                invalid = df[~df[col].astype(str).str.match(pat)]
                if len(invalid) > 0:
                    STATUS_FLAG = -1
                print(f"Table {tbl} | Invalid {col}: {len(invalid)}")
                logging.info(f"Table {tbl} | Invalid {col}: {len(invalid)}")

    conn.close()
    print(f"\n📌 FINAL STATUS = {STATUS_FLAG}")
    logging.info(f"FINAL STATUS = {STATUS_FLAG}")
    return STATUS_FLAG


if __name__ == "__main__":
    run_validation()
    print("\n📌 Validation Finished — Check logs!")



🔎 RUNNING FULL VALIDATION...

🧾 Tables in DB:
                     name
0        sqlite_sequence
1         fact_ivf_cycle
2   fact_transfer_embryo
3             dim_female
4               dim_male
5           dim_protocol
6             dim_doctor
7            dim_outcome
8             dim_embryo
9               dim_time
10         fact_transfer 

🔍 Duplicates in sqlite_sequence → 0
🔍 Duplicates in fact_ivf_cycle → 0
🔍 Duplicates in fact_transfer_embryo → 0
🔍 Duplicates in dim_female → 0
🔍 Duplicates in dim_male → 0
🔍 Duplicates in dim_protocol → 0
🔍 Duplicates in dim_doctor → 0
🔍 Duplicates in dim_outcome → 0
🔍 Duplicates in dim_embryo → 0
🔍 Duplicates in dim_time → 0
🔍 Duplicates in fact_transfer → 0

🕳 NULL VALUES CHECK:

⚠ Table: sqlite_sequence
name    0
seq     0
dtype: int64

⚠ Table: fact_ivf_cycle
cycle_sk                 0
case_id                  0
female_id                0
male_id                  0
protocol_id              0
doctor_id                0
outcome_id          